In [1]:
import sys
import os
sys.path.append(os.path.dirname(os.path.abspath("./")))
from src.models.ollama_model import OllamaModel
from src.tools.base import Tool

In [2]:
model_service = OllamaModel(
    model="mistral-nemo:latest",
    system_prompt="You are a helpful assistant!",
    is_tool_use_model=True
)

In [3]:
# check the generate method
response = model_service.generate_text_api_call("Who is the Prime minister of India?")

REQUEST RESPONSE: 200


Response from OllamaModel::mistral-nemo:latest={'name': 'Narendra Modi', 'position': 'Prime Minister of India', 'country': 'India'}


In [4]:
def add(num1: int, num2: int) -> int:
    "adds two int"
    return num1 + num2


add_tool = Tool.from_function(function=add, return_direct=False)

In [5]:
response = model_service.chat({"role": "user", "content": "What is 3789 + 1024?"}, tools=[add_tool])

[{'role': 'system', 'content': 'You are a helpful assistant!'}, {'role': 'user', 'content': 'What is 3789 + 1024?'}]
{'model': 'mistral-nemo:latest', 'created_at': '2024-09-13T13:44:25.55098272Z', 'message': {'role': 'assistant', 'content': 'The sum of 3789 and 1024 is 4813.'}, 'done_reason': 'stop', 'done': True, 'total_duration': 437436089, 'load_duration': 11862182, 'prompt_eval_count': 52, 'prompt_eval_duration': 23468000, 'eval_count': 22, 'eval_duration': 277290000}


In [6]:
response

{'role': 'assistant', 'content': 'The sum of 3789 and 1024 is 4813.'}

## Test Agent

In [7]:
from src.agents.function_calling_agent import FunctionCallingAgent

function_calling_agent = FunctionCallingAgent(
    model_name='llama3.1:latest',
    model_service="ollama",
    tools=[add_tool],
    stop_token=None,
    system_prompt="You are a helpful assistant!"
)

In [8]:
function_calling_agent.chat("What is 3789 + 1024?", max_retries=3)

[{'role': 'system', 'content': 'You are a helpful assistant!'}, {'role': 'user', 'content': 'What is 3789 + 1024?'}]
{'model': 'llama3.1:latest', 'created_at': '2024-09-13T13:44:35.136674384Z', 'message': {'role': 'assistant', 'content': '', 'tool_calls': [{'function': {'name': 'add', 'arguments': {'a': '3789', 'b': '1024'}}}]}, 'done_reason': 'stop', 'done': True, 'total_duration': 374747130, 'load_duration': 10307577, 'prompt_eval_count': 146, 'prompt_eval_duration': 29867000, 'eval_count': 24, 'eval_duration': 211600000}
Encountered Validation Error, retrying execution: 2 validation errors for add
num1
  Field required [type=missing, input_value={'a': '3789', 'b': '1024'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/missing
num2
  Field required [type=missing, input_value={'a': '3789', 'b': '1024'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/missing
[{'role': 'system', 'content': 'You are a helpful as

'The result of the calculation is 4813.'